In [1]:
!pip install -q kaggle

from google.colab import files
files.upload()  # Upload the kaggle.json file

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sowaadrahman","key":"ce2fab7c1ebaf7fc9160542dc7722a29"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d requiemonk/sentinel12-image-pairs-segregated-by-terrain


Dataset URL: https://www.kaggle.com/datasets/requiemonk/sentinel12-image-pairs-segregated-by-terrain
License(s): Attribution 4.0 International (CC BY 4.0)
100% 2.54G/2.55G [00:17<00:00, 160MB/s]
100% 2.55G/2.55G [00:17<00:00, 153MB/s]


In [4]:
import os
import zipfile

# Path to the downloaded ZIP file
zip_file_path = "sentinel12-image-pairs-segregated-by-terrain.zip"

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('dataset')

print("Extraction complete.")

# Delete the ZIP file after extraction
if os.path.exists(zip_file_path):
    os.remove(zip_file_path)
    print(f"{zip_file_path} deleted.")
else:
    print(f"{zip_file_path} not found.")


Extraction complete.
sentinel12-image-pairs-segregated-by-terrain.zip deleted.


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
dataset_path = '/content/dataset/v_2'  # Adjust the path as needed

# Create an ImageDataGenerator instance for loading images in batches
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

# Define training and validation data generators
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),  # Resize images
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use this for training
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),  # Resize images
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use this for validation
)


Found 25600 images belonging to 4 classes.
Found 6400 images belonging to 4 classes.


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define image dimensions
IMAGE_SIZE = (224, 224)  # Standard size for most pre-trained models
BATCH_SIZE = 32

# Path to dataset
dataset_path = "/content/dataset/v_2"

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% of data for validation
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Number of classes
num_classes = train_generator.num_classes

# Function to add classification layers to the pre-trained model
def build_model(base_model):
    # Freeze the convolutional base
    base_model.trainable = False

    # Add classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Load and compile pre-trained models

# 1. EfficientNetB0
efficientnet_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
efficientnet_model = build_model(efficientnet_base)

# 2. InceptionV3
inceptionv3_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
inceptionv3_model = build_model(inceptionv3_base)

# Train the models

# Training EfficientNetB0 model
print("Training EfficientNetB0 model...")
efficientnet_model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Training InceptionV3 model
print("Training InceptionV3 model...")
inceptionv3_model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Evaluate the models

# Evaluate EfficientNetB0
efficientnet_eval = efficientnet_model.evaluate(validation_generator)
print(f"EfficientNetB0 Evaluation - Loss: {efficientnet_eval[0]}, Accuracy: {efficientnet_eval[1]}")

# Evaluate InceptionV3
inceptionv3_eval = inceptionv3_model.evaluate(validation_generator)
print(f"InceptionV3 Evaluation - Loss: {inceptionv3_eval[0]}, Accuracy: {inceptionv3_eval[1]}")


Found 25600 images belonging to 4 classes.
Found 6400 images belonging to 4 classes.
Training EfficientNetB0 model...
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


800/800 ━━━━━━━━━━━━━━━━━━━━ 2641s 3s/step - accuracy: 0.2495 - loss: 1.3890 - val_accuracy: 0.2500 - val_loss: 1.3898
Epoch 2/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 2961s 4s/step - accuracy: 0.2549 - loss: 1.3877 - val_accuracy: 0.2500 - val_loss: 1.3875
Epoch 3/10
134/800 ━━━━━━━━━━━━━━━━━━━━ 35:04 3s/step - accuracy: 0.2518 - loss: 1.3865